In [89]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

In [174]:
# import data
data=pd.read_csv('refugee_data/refugee_data_final.csv')
data = data[data['country']!='Russian Federation']

In [175]:
data["historic_gdp_millions"]

0     109591.707802
1      82017.743416
2        860.521119
3      13760.513969
4       2904.732773
          ...      
58     45599.994005
59    201175.469114
60     10224.897935
61     33728.622819
62     12953.535496
Name: historic_gdp_millions, Length: 62, dtype: float64

In [176]:
# if we want to recalc percent total of conflict after dropping Russia...
data['pct_tot'] = data['individualPerCountry'] / data.groupby('conflict')['individualPerCountry'].transform('sum')

In [178]:
data['bilateral_migration_percap'] = data['bilateral_migration']/data['population']

In [179]:
data['gdp_per_cap'] = data['gdp_millions']*1000000/data['population']

In [180]:
data['migrants_per_cap'] = data['total_recored_migrants']/data['population']

In [181]:
cols_to_scale = ["historic_gdp_millions",'population', 'remittances']

In [182]:
scaler = MinMaxScaler()
for col in cols_to_scale:
    print(col)
    normed = pd.DataFrame()
    
    for y, x in data.groupby('conflict'):
        norm_ = [i[0] for i in scaler.fit_transform(x[col].values.reshape(-1,1))]
        countries = x['country']
        conflict_ = x['conflict']
        res = pd.DataFrame(tuple(zip(countries,conflict_,norm_)), columns=['country','conflict',f"{col}_norm"])
        normed = normed.append(res)
    data = pd.merge(data, normed, left_on=['country','conflict'], right_on=['country','conflict'], how='right')

v2x_libdem
gdp_millions
historic_gdp_millions
gdp_per_cap_historic


<ipython-input-182-cf56ce5f6369>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  normed = normed.append(res)
<ipython-input-182-cf56ce5f6369>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  normed = normed.append(res)
<ipython-input-182-cf56ce5f6369>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  normed = normed.append(res)
<ipython-input-182-cf56ce5f6369>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  normed = normed.append(res)
<ipython-input-182-cf56ce5f6369>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  normed = normed.append(res)


population
remittances


<ipython-input-182-cf56ce5f6369>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  normed = normed.append(res)
<ipython-input-182-cf56ce5f6369>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  normed = normed.append(res)
<ipython-input-182-cf56ce5f6369>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  normed = normed.append(res)
<ipython-input-182-cf56ce5f6369>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  normed = normed.append(res)
<ipython-input-182-cf56ce5f6369>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  normed = normed.append(res)


In [183]:
ukr = data[data['conflict']=='Ukraine']

In [184]:
#train model without Ukraine so we use withoutUkrain
withoutUkrainData=data[(data["Ukraine"]!=1) & (data['touching']==1)]

In [185]:
# set y variable we are trying to predict
y=withoutUkrainData['pct_tot']

In [193]:
# set independant variables 
features_cols = [
    "historic_gdp_millions_norm",
    "v2x_libdem"
#                    'gdp_per_cap_historic_norm',
#                    'gdp_millions_norm'
#                  'bilateral_migration_norm', 
#                  'gdp_per_cap',
#                 'migrants_per_cap',
#                  'population_norm', 
#                 'v2x_libdem',
#                  'v2xeg_eqdr',
#                  'same_language',
#                  'migrant_ratio',
#                  'remittances_norm'
]
features_normalized = withoutUkrainData[features_cols]

In [194]:
ukr[['country']+features_cols]

,country,historic_gdp_millions_norm,v2x_libdem
46,Hungary,0.246093,0.362
47,Moldova,0.000000,0.615
48,Poland,1.000000,0.413
49,Romania,0.404988,0.644
50,Slovakia,0.159493,0.769


###   We also tried to use these additional features in the model, however none of these features had a high enough p value to be included in the final model
 1. "normalized_qrdp"
 2. "same_language"
 3. "touching"
 4. "normalized_lib"
 5. "normalized_pop"
 6. "normalized_migr_ratio"
 7. "normalized_gdp"
 8. "normalized_export_trade"
 9. "normalized_remittances"
 

In [195]:
# Run the linear regression. 
import statsmodels.api as sm

In [196]:
results=sm.OLS(y,features_normalized).fit()

In [197]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                pct_tot   R-squared (uncentered):                   0.592
Model:                            OLS   Adj. R-squared (uncentered):              0.571
Method:                 Least Squares   F-statistic:                              27.59
Date:                Fri, 15 Apr 2022   Prob (F-statistic):                    3.98e-08
Time:                        13:36:17   Log-Likelihood:                          12.209
No. Observations:                  40   AIC:                                     -20.42
Df Residuals:                      38   BIC:                                     -17.04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
historic_gdp_millions_norm     0.2555      0.071      3.617      0.001       0.112       0.398
v2x_libdem                     0.3871      0.139      2.775      0.009       0.105       0.670
==============================================================================
Omnibus:                        3.088   Durbin-Watson:                   2.339
Prob(Omnibus):                  0.213   Jarque-Bera (JB):                2.191
Skew:                          -0.191   Prob(JB):                        0.334
Kurtosis:                       4.081   Cond. No.                         2.84
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [198]:
features_to_predict=data[features_cols]
shares = results.predict(features_to_predict)
data['predicted_shares'] = shares
ukr_results = data[data['Ukraine']==1][['country','pct_tot','predicted_shares']]
ukr_results.to_csv('outputs/ukraine_model_results.csv',index=False)

In [199]:
ukr_results

,country,pct_tot,predicted_shares
46,Hungary,0.094222,0.202999
47,Moldova,0.093682,0.238070
48,Poland,0.586410,0.415333
49,Romania,0.154561,0.352753
50,Slovakia,0.071125,0.338428
